In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/Research1_code/

In [ ]:
cd mnist/CNN-2

In [4]:
import numpy as np
import sys, os, random
import matplotlib.pyplot as plt
import pickle, gzip
from tqdm import tqdm,tqdm_notebook
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
batch_size = 128
seed_num = 81

# For reproducibility when you run the file with .py
torch.cuda.is_available()
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True

torch.backends.cudnn.deterministic =True

In [6]:
# Data Augmentation 
train_transform = transforms.Compose([transforms.RandomRotation(30), transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.1307,],[0.3081,])])
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.1307,],[0.3081,])])

# Splitting the training and test datasets
train_data = datasets.MNIST(os.getcwd(), train=True,
                              download=True, transform=train_transform)
test_data = datasets.MNIST(os.getcwd(), train=False,
                             download=True, transform=test_transform)

In [7]:
# Split the training set indices into training and validation set indices using 80:20 ratio
np.random.seed(seed_num)
len_trainset = len(train_data)
index_list = list(range(len_trainset))
np.random.shuffle(index_list)
split_index = 50000
train_indices, valid_indices =  index_list[:split_index], index_list[split_index:]

# Creating Samplers for training and validation set using the indices
np.random.seed(seed_num)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

torch.manual_seed(seed_num)

train_iterator = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_iterator = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [8]:
# CNN Model

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0, bias= True)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0, bias= True)
        
        # Fully connected 1
        self.fc1 = nn.Linear(32 * 5 * 5, 10,bias= True) 
    
    def forward(self, x):
        # Set 1
        out = F.max_pool2d(F.relu(self.cnn1(x)),2)
        
        # Set 2
        out = F.max_pool2d(F.relu(self.cnn2(out)),2) 
        
        # Flatten
        out = out.view(out.size(0), -1)

        # Dense
        out = self.fc1(out)
        
        return out

In [9]:
torch.manual_seed(seed_num)
unit=128

# Summary
model = CNNModel()
print("Model:\n",model)

Model:
 CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=10, bias=True)
)


In [10]:
# Layer names
layer_name = [n for n, p in model.named_parameters()]
layer_name

['cnn1.weight',
 'cnn1.bias',
 'cnn2.weight',
 'cnn2.bias',
 'fc1.weight',
 'fc1.bias']

In [11]:
# Loading the weights of ternary model 
model = torch.load('CNN_2_quant.pt')
model = model.cuda()
print("Loading weights done !")

# Total number of ternary weights (+w, -w)
totalParams = 0
for i in layer_name:
  print(i,(model.state_dict()[i] !=0).sum())
  totalParams +=  (model.state_dict()[i] !=0).sum()
    
print("Total Parameters:",totalParams, '\n')

Loading weights done !
cnn1.weight tensor(104, device='cuda:0')
cnn1.bias tensor(13, device='cuda:0')
cnn2.weight tensor(436, device='cuda:0')
cnn2.bias tensor(28, device='cuda:0')
fc1.weight tensor(780, device='cuda:0')
fc1.bias tensor(9, device='cuda:0')
Total Parameters: tensor(1370, device='cuda:0') 



In [13]:
# Model's performance on test set

correct_count, all_count = 0, 0
for images,labels in test_iterator:
      for image,label in zip(images,labels):

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Test Accuracy =", (correct_count/(all_count)))


Number Of Images Tested = 10000

Model Test Accuracy = 0.893


In [14]:
# For each layer, model's ternary weights
state_dict = model.state_dict()

layer_distinct_weights = {}

for i in layer_name:
  imd = torch.unique(model.state_dict()[i])
  print(i+ ' hidden layer dimension', model.state_dict()[i].shape)
  print("Unique values of weight in "+ i+ " th hidden layer : ", imd)
  layer_distinct_weights[i] = imd.cpu().numpy().tolist()
  print()

cnn1.weight hidden layer dimension torch.Size([16, 1, 3, 3])
Unique values of weight in cnn1.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn1.bias hidden layer dimension torch.Size([16])
Unique values of weight in cnn1.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn2.weight hidden layer dimension torch.Size([32, 16, 3, 3])
Unique values of weight in cnn2.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn2.bias hidden layer dimension torch.Size([32])
Unique values of weight in cnn2.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

fc1.weight hidden layer dimension torch.Size([10, 800])
Unique values of weight in fc1.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

fc1.bias hidden layer dimension torch.Size([10])
Unique values of weight in fc1.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')



In [15]:
keys = list(state_dict.keys())
print(keys)

['cnn1.bias', 'cnn1.weight', 'cnn2.bias', 'cnn2.weight', 'fc1.bias', 'fc1.weight']


In [17]:
for i in keys:
  print(i, state_dict[i].shape)

cnn1.bias torch.Size([16])
cnn1.weight torch.Size([16, 1, 3, 3])
cnn2.bias torch.Size([32])
cnn2.weight torch.Size([32, 16, 3, 3])
fc1.bias torch.Size([10])
fc1.weight torch.Size([10, 800])


In [21]:
# Duplicate architecture of the CNN Model

class CNNModel1(nn.Module):
    def __init__(self, dn_info):
        super(CNNModel1, self).__init__()

        self.dn_info = dn_info       # Dead Neuron info
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0, bias= True)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0, bias= True)
        
        # Fully connected 1
        self.fc1 = nn.Linear(32 * 5 * 5, 10,bias= True) 
    
    def forward(self, x):
        # Set 1
        out = F.max_pool2d(F.relu(self.cnn1(x)),2)
        
        # Set 2
        out = F.max_pool2d(F.relu(self.cnn2(out)),2) 
        
        # Flatten
        out = out.view(out.size(0), -1)

        ####################################
        # Storing dead neurons indices
        idx1 = torch.where(out.cpu() == 0.)[1]
        
        for j in idx1:
            self.dn_info[str(j.item())] += 1
            
        #####################################

        # Dense
        out = self.fc1(out)
        
        return out

In [22]:
def getDeadN_info(dn_info, unit, state_dict, ds, nameOftheSet):

  for i in range(unit):
      dn_info[str(i)] = 0
      
  model1 = CNNModel1(dn_info)
  model1 = model1.cuda()

  model1.load_state_dict(state_dict)

  correct_count, all_count = 0, 0
  model1.eval()

  for images,labels in ds:
        for image,label in zip(images,labels):

          if torch.cuda.is_available():
              img = image.cuda()
              lab = label.cuda()
              img = img[None,].type('torch.cuda.FloatTensor')

          with torch.no_grad():
              output_ = model1(img) 

          pred_label = output_.argmax()

          if(pred_label.item()==lab.item()):
            correct_count += 1
          all_count += 1

  print("Number Of Images =", all_count)
  print(f"Model {nameOftheSet} Accuracy =", (correct_count/(all_count)))

In [23]:
dn_info_train = {}
dn_info_val = {}
dn_info_test = {}

getDeadN_info(dn_info= dn_info_train, unit = 800, state_dict = state_dict, ds= train_iterator, nameOftheSet = "Training")
getDeadN_info(dn_info= dn_info_val, unit = 800, state_dict = state_dict, ds= val_iterator, nameOftheSet = "Validation")
getDeadN_info(dn_info= dn_info_test, unit = 800, state_dict = state_dict, ds= test_iterator, nameOftheSet = "Test")

Number Of Images = 50000
Model Training Accuracy = 0.8528
Number Of Images = 10000
Model Validation Accuracy = 0.8465
Number Of Images = 10000
Model Test Accuracy = 0.893


In [24]:
max_dn_val = max(dn_info_val.values())
max_dn_test = max(dn_info_test.values())
max_dn_train = max(dn_info_train.values())
max_dn_train, max_dn_val, max_dn_test

(50000, 10000, 10000)

In [25]:
dead_n_idx = [] 

for i, j in dn_info_train.items():
  if j == max_dn_train:
    dead_n_idx.append(i)

In [26]:
print("FC Layer :",len(dead_n_idx)) # Number of neurons that are dead

FC Layer : 75


In [27]:
print(dead_n_idx) # Indices of neuron that are dead

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '525', '526', '527', '528', '529', '530', '531', '532', '533', '534', '535', '536', '537', '538', '539', '540', '541', '542', '543', '544', '545', '546', '547', '548', '549', '775', '776', '777', '778', '779', '780', '781', '782', '783', '784', '785', '786', '787', '788', '789', '790', '791', '792', '793', '794', '795', '796', '797', '798', '799']


In [28]:
state_dict1 = state_dict

In [29]:
layer_distinct_weights

{'cnn1.bias': [-1.0, 0.0, 1.0],
 'cnn1.weight': [-1.0, 0.0, 1.0],
 'cnn2.bias': [-1.0, 0.0, 1.0],
 'cnn2.weight': [-1.0, 0.0, 1.0],
 'fc1.bias': [-1.0, 0.0, 1.0],
 'fc1.weight': [-1.0, 0.0, 1.0]}

In [31]:
total_trans = 0
layers_name = ['fc1.weight']

for l in layers_name:
  print(l)
  z = state_dict1[l]

  if len(layer_distinct_weights[l]) > 2 :
    w_neg, w_0, w_pos =  layer_distinct_weights[l]
    print(w_neg, w_0, w_pos)
  else:
    w_neg, w_pos = layer_distinct_weights[l]
    print(w_neg,  w_pos)

  for idx in dead_n_idx:
      imd = z[:,eval(idx)]
      trans = torch.where(imd == w_neg)[0].nelement() + torch.where(imd == w_pos)[0].nelement()
      total_trans += trans * 2
    
print("Total dead transitions for the output layer :", total_trans)
    
    

fc1.weight
-1.0 0.0 1.0
Total dead transitions for the output layer : 20


In [ ]:
# Fault coverage that we have obtained from main file : 2471 / 2740

In [32]:
# Adding more 60 transitions to the numerator will make the net fault coverage
(2471 + 20)/2740

0.9091240875912409